In [1]:
# Debe ser la PRIMERA celda ejecutada en la sesión
import sys, asyncio
if sys.platform.startswith("win"):
    try:
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
        print("Policy:", type(asyncio.get_event_loop_policy()).__name__)
    except Exception as e:
        print("No pude fijar policy:", e)


Policy: WindowsProactorEventLoopPolicy


In [2]:
import asyncio, sys

async def _probe():
    proc = await asyncio.create_subprocess_exec(sys.executable, "-c", "print('ok')",
                                                stdout=asyncio.subprocess.PIPE)
    out, _ = await proc.communicate()
    print(out.decode().strip())

await _probe()


NotImplementedError: 

In [ ]:
# FIX para Jupyter en Windows: usar ProactorEventLoop (soporta subprocess)
import sys, asyncio
if sys.platform.startswith("win"):
    asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())


Configuración e imports

In [ ]:
# 1) Configuración e imports
from __future__ import annotations

import re
from typing import Dict, List, Tuple, Optional
import pandas as pd

from playwright.sync_api import sync_playwright

DEFAULT_WAIT_MS = 60_000
DEFAULT_SLOW_MO_MS = 0  # Subí a 300 si querés ver paso a paso

def _clean(s: Optional[str]) -> str:
    if s is None:
        return ""
    return str(s).replace("\r", "").replace("\n", " ").strip()

def _split_domicilio_3lineas(txt: str) -> Tuple[str, str, str]:
    """
    Domicilio multilínea → (domicilio, localidad, provincia).
    Si no viene en 3 líneas, devuelve lo que tenga y el resto vacío.
    """
    if not txt:
        return ("", "", "")
    lines = [l.strip() for l in txt.replace("\r","").split("\n") if l.strip()]
    dom = lines[0] if len(lines)>0 else ""
    loc = lines[1] if len(lines)>1 else ""
    prov= lines[2] if len(lines)>2 else ""
    return (dom, loc, prov)


2 Acceso y autenticación (OT + credenciales)

In [ ]:
# 2) Acceso y autenticación (versión ASYNC)
import re
from playwright.async_api import async_playwright

DEFAULT_WAIT_MS = 60_000
DEFAULT_SLOW_MO_MS = 0

async def login_y_abrir_ot_async(context, usuario: str, password: str, ot_numero: str):
    page = await context.new_page()
    # Login
    await page.goto("https://app.inti.gob.ar/MetroWeb/pages/ingreso.jsp", timeout=DEFAULT_WAIT_MS)
    await page.wait_for_load_state("domcontentloaded", timeout=DEFAULT_WAIT_MS)

    usr = page.locator('input[name="usuario"]') or page.locator("#usuario")
    if not await usr.count():
        usr = page.locator('input[type="text"]').first
    pwd = page.locator('input[name="contrasena"]') or page.locator('input[name="password"]')
    if not await pwd.count():
        pwd = page.locator('input[type="password"]').first

    await usr.fill(usuario)
    await pwd.fill(password)
    if await page.locator('input[value="Ingresar"]').count():
        await page.click('input[value="Ingresar"]')
    else:
        await page.keyboard.press("Enter")
    await page.wait_for_load_state("networkidle", timeout=DEFAULT_WAIT_MS)

    # Buscar OT
    await page.goto("https://app.inti.gob.ar/MetroWeb/entrarPML.do", timeout=DEFAULT_WAIT_MS)
    await page.wait_for_selector('input[name="numeroOT"]', timeout=DEFAULT_WAIT_MS)
    await page.fill('input[name="numeroOT"]', ot_numero)
    if await page.locator('input[value="Buscar"]').count():
        await page.click('input[value="Buscar"]')
    else:
        await page.keyboard.press("Enter")
    await page.wait_for_load_state("networkidle", timeout=DEFAULT_WAIT_MS)

    # Abrir trámite VPE
    link_vpe = page.locator('a[href*="tramiteVPE"]').first
    if not await link_vpe.count():
        raise RuntimeError(f"No se encontró VPE para la OT {ot_numero}.")
    vpe_text = _clean(await link_vpe.inner_text())
    await link_vpe.click()
    await page.wait_for_load_state("networkidle", timeout=DEFAULT_WAIT_MS)

    # Forzar detalle.jsp
    if "acc=resumen" in page.url or page.url.endswith("/resumen.jsp"):
        await page.goto(page.url.replace("acc=resumen","acc=detalle").replace("/resumen.jsp","/detalle.jsp"),
                        timeout=DEFAULT_WAIT_MS)

    return page, vpe_text


Extracción de datos generales (cabecera + domicilio instalación)

In [ ]:
# 3) Datos generales (versión ASYNC)
async def td_value_any_async(page, labels, keep_newlines: bool=False) -> str:
    for lbl in labels:
        loc = page.locator(f"xpath=//td[contains(normalize-space(.), '{lbl}')]/following-sibling::td[1]")
        if await loc.count():
            txt = await loc.first.inner_text()
            if not keep_newlines:
                txt = txt.replace("\r","").replace("\n"," ")
            return txt.strip()
    return ""

async def leer_generales_async(page):
    out = {}
    out["Razon_social_propietario"] = await td_value_any_async(
        page, ["Nombre del Usuario del instrumento", "Usuario del equipo", "Usuario del instrumento"]
    ) or await td_value_any_async(page, ["Usuario Representado"])
    out["CUIT"] = await td_value_any_async(page, ["Nº de CUIT","N° de CUIT","CUIT"])

    out["Direccion_legal"] = await td_value_any_async(page, ["Dirección Legal","Domicilio Legal","Direcci\u00f3n Legal"], keep_newlines=True)

    dom_ml = await td_value_any_async(page, ["Domicilio donde están", "Domicilio donde est\u00e1n", "Domicilio"], keep_newlines=True)
    dom, loc, prov = _split_domicilio_3lineas(dom_ml)
    out["Instalacion_domicilio"] = dom
    out["Instalacion_localidad"] = loc
    out["Instalacion_provincia"] = prov

    out["Fecha_verificacion"] = await td_value_any_async(
        page, ["Fecha de Verificación","Fecha verificaci\u00f3n","Fecha última Verificación","Fecha &uacute;ltima Verificaci\u00f3n"]
    )
    out["Tipo_verificacion"] = await td_value_any_async(page, ["Tipo de Verificación","Tipo Verificación","Tipo verificación"])
    out["Tolerancia"] = await td_value_any_async(page, ["Tolerancia"])
    return out


Extracción del instrumento: separar Receptor vs Indicador (grilla por fila/columna)

In [ ]:
# 4) Receptor / Indicador (versión ASYNC)
async def grid_val_async(page, row_label_regex: str, col_header_regex: str) -> str:
    tablas = page.locator("xpath=//table[.//td[contains(.,'Receptor') or contains(.,'Indicador')]]")
    for t in range(await tablas.count()):
        tab = tablas.nth(t)
        headers = tab.locator("xpath=.//tr[1]/td|.//tr[1]/th")
        idx = {}
        for i in range(await headers.count()):
            h = _clean(await headers.nth(i).inner_text())
            if re.search(col_header_regex, h, re.I):
                idx["col"] = i+1
        if "col" not in idx:
            continue

        filas = tab.locator("xpath=.//tr[position()>1]")
        for r in range(await filas.count()):
            first_td = _clean(await filas.nth(r).locator("xpath=./td[1]").inner_text())
            if re.search(row_label_regex, first_td, re.I):
                val = filas.nth(r).locator(f"xpath=./td[{idx['col']}]")
                return _clean(await val.inner_text())
    return ""

async def leer_receptor_indicador_async(page):
    receptor = {
        "Fabricante": "", "Marca": "", "Modelo": "", "Nro_serie": "",
        "Cod_aprob_mod": "", "Origen": "",
        "Nro_aprob_mod": "", "Fecha_aprob_mod": ""
    }
    indicador = {
        "Marca": "", "Modelo": "", "Nro_serie": "",
        "Cod_aprob_mod": "", "Origen": "",
        "Nro_aprob_mod": "", "Fecha_aprob_mod": ""
    }

    for col_pat, target in [
        (r"Receptor", receptor),
        (r"Indicador", indicador),
    ]:
        target["Marca"]         = await grid_val_async(page, r"^\s*Marca\s*:?$", col_pat)
        target["Modelo"]        = await grid_val_async(page, r"^\s*Modelo\s*:?$", col_pat)
        target["Nro_serie"]     = await grid_val_async(page, r"(N.?°|Nro|Número)\s*de\s*serie", col_pat)
        target["Cod_aprob_mod"] = await grid_val_async(page, r"(C(ó|o)d(igo)?\.?\s*(de\s*)?aprobaci(ó|o)n\s*de\s*modelo|BF-|AM-)", col_pat)
        target["Origen"]        = await grid_val_async(page, r"^\s*Origen\s*:?$", col_pat)
        target["Nro_aprob_mod"] = await grid_val_async(page, r"^\s*N.?°\s*de\s*Aprobaci(ó|o)n\s*de\s*modelo\s*:?$", col_pat)
        target["Fecha_aprob_mod"]=await grid_val_async(page, r"^\s*Fecha\s*de\s*Aprobaci(ó|o)n\s*de\s*modelo\s*:?$", col_pat)

    return receptor, indicador


Características metrológicas (del modelo). Fallback: si no aparece dd=dt, usar e

In [ ]:
# 5) Características metrológicas (versión ASYNC)
async def abrir_detalle_modelo_async(context, page):
    link = page.locator('a[href*="modeloDetalle.do"]').first
    if not await link.count():
        link = page.locator('a[href*="modelo"]').first
    if not await link.count():
        return None
    href = await link.get_attribute("href") or ""
    if not href:
        return None
    p2 = await context.new_page()
    await p2.goto(href if href.startswith("http") else "https://app.inti.gob.ar"+href, timeout=DEFAULT_WAIT_MS)
    await p2.wait_for_load_state("networkidle", timeout=DEFAULT_WAIT_MS)
    return p2

async def td_exact_async(page, lbls):
    for lbl in lbls:
        loc = page.locator(f"xpath=//td[normalize-space(.)='{lbl}']/following-sibling::td[1]")
        if await loc.count():
            return _clean(await loc.first.inner_text())
    return ""

async def leer_caracteristicas_metrologicas_async(context, page):
    out = {"Max": "", "Min": "", "e": "", "dd_dt": "", "Clase": ""}
    p2 = await abrir_detalle_modelo_async(context, page)
    if p2 is None:
        return out
    try:
        out["Max"]   = await td_exact_async(p2, ["Máximo","Maximo"])
        out["Min"]   = await td_exact_async(p2, ["Mínimo","Minimo"])
        out["e"]     = await td_exact_async(p2, ["e"])
        out["Clase"] = await td_exact_async(p2, ["Clase"])
        out["dd_dt"] = await td_exact_async(p2, ["d","dd=dt","dd = dt"]) or out["e"]
    finally:
        try: await p2.close()
        except Exception: pass
    return out


Extracción integral por OT (une pasos 2→5 y levanta cada instrumento del VPE)

In [ ]:
# 6) Extracción integral por OT (versión ASYNC)
async def extraer_camiones_por_ot_async(ot_numero: str, usuario: str, password: str, mostrar_navegador: bool=True) -> pd.DataFrame:
    filas = []
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=not mostrar_navegador, slow_mo=DEFAULT_SLOW_MO_MS)
        context = await browser.new_context()
        page, vpe_text = await login_y_abrir_ot_async(context, usuario, password, ot_numero)

        html = await page.content()
        ids = re.findall(r'name="instrumentos\[\d+\]\.idInstrumento"\s+value="(\d+)"', html)
        if not ids:
            raise RuntimeError("No se encontraron instrumentos en el detalle del VPE.")

        for idx, id_inst in enumerate(ids, start=1):
            det = await context.new_page()
            try:
                await det.goto(f"https://app.inti.gob.ar/MetroWeb/instrumentoDetalle.do?idInstrumento={id_inst}", timeout=DEFAULT_WAIT_MS)
                await det.wait_for_load_state("networkidle", timeout=DEFAULT_WAIT_MS)

                gen   = await leer_generales_async(det)
                rec, ind = await leer_receptor_indicador_async(det)
                cm    = await leer_caracteristicas_metrologicas_async(context, det)

                fila = {
                    "OT": ot_numero,
                    "VPE": vpe_text,
                    "Fecha verificación": gen["Fecha_verificacion"],
                    "Instalación - Domicilio": gen["Instalacion_domicilio"],
                    "Instalación - Localidad": gen["Instalacion_localidad"],
                    "Instalación - Provincia": gen["Instalacion_provincia"],
                    "Razón Social (Propietario)": gen["Razon_social_propietario"],
                    "CUIT": gen["CUIT"],
                    "Dirección Legal (Fiscal)": gen["Direccion_legal"],
                    "Tipo de Verificación": gen["Tipo_verificacion"],
                    "Tolerancia": gen["Tolerancia"],

                    "Fabricante receptor": rec["Fabricante"] or "",
                    "Marca Receptor": rec["Marca"],
                    "Modelo Receptor": rec["Modelo"],
                    "N° de serie Receptor": rec["Nro_serie"],
                    "Cód ap. mod. Receptor": rec["Cod_aprob_mod"],
                    "Origen Receptor": rec["Origen"],

                    "e": cm["e"], "máx": cm["Max"], "mín": cm["Min"], "dd=dt": cm["dd_dt"], "clase": cm["Clase"],

                    "Marca Indicador": ind["Marca"],
                    "Modelo Indicador": ind["Modelo"],
                    "N° de serie Indicador": ind["Nro_serie"],
                    "Código ap. mod. Indicador": ind["Cod_aprob_mod"],
                    "Origen Indicador": ind["Origen"],
                    "N° Aprobación Modelo (Ind.)": ind["Nro_aprob_mod"],
                    "Fecha Aprobación Modelo (Ind.)": ind["Fecha_aprob_mod"],
                }
                filas.append(fila)
                print(f"✅ Instrumento {idx} capturado (Receptor {fila['Marca Receptor']} / Indicador {fila['Marca Indicador']})")

            finally:
                try: await det.close()
                except Exception: pass

        await browser.close()

    cols = [
        "OT","VPE","Fecha verificación",
        "Instalación - Domicilio","Instalación - Localidad","Instalación - Provincia",
        "Razón Social (Propietario)","CUIT","Dirección Legal (Fiscal)",
        "Tipo de Verificación","Tolerancia",
        "Fabricante receptor","Marca Receptor","Modelo Receptor","N° de serie Receptor","Cód ap. mod. Receptor","Origen Receptor",
        "e","máx","mín","dd=dt","clase",
        "Marca Indicador","Modelo Indicador","N° de serie Indicador","Código ap. mod. Indicador","Origen Indicador",
        "N° Aprobación Modelo (Ind.)","Fecha Aprobación Modelo (Ind.)",
    ]
    df = pd.DataFrame(filas)
    for c in cols:
        if c not in df.columns: df[c] = ""
    return df[cols]


Hoja de verificación previa (para revisar que esté todo)

In [ ]:
# 7) Hoja de verificación previa (sin tocar tu Excel definitivo)
def _parse_fecha_dd_mmm_aaaa(fecha_txt: str) -> Tuple[str,str,str]:
    """
    '15 de octubre de 2025' → ('15','octubre','2025')
    Si no matchea, devuelve ('','','').
    """
    m = re.search(r"(\d{1,2})\s+de\s+([A-Za-záéíóúñ]+)\s+de\s+(\d{4})", fecha_txt, re.I)
    if not m:
        return ("", "", "")
    return (m.group(1), m.group(2).lower(), m.group(3))

def armar_hoja_verificacion(df: pd.DataFrame) -> pd.DataFrame:
    # Desglosar fecha en día/mes/año (para que visualmente copie tu planilla)
    dia, mes, anio = [], [], []
    for f in df["Fecha verificación"].fillna(""):
        d,m,a = _parse_fecha_dd_mmm_aaaa(f)
        dia.append(d); mes.append(m); anio.append(a)

    out = pd.DataFrame({
        "Número de O.T.": df["OT"],
        "VPE Nº": df["VPE"],
        "día (n°)": dia,
        "mes (texto)": mes,
        "año (n°)": anio,

        "Razón social (Propietario)": df["Razón Social (Propietario)"],
        "Domicilio (Fiscal)": df["Dirección Legal (Fiscal)"],  # si después querés desdoblar en Loc/Prov fiscal, lo agregamos
        "Localidad (Fiscal)": "",
        "Provincia (Fiscal)": "",

        "Lugar propio de instalación - Domicilio": df["Instalación - Domicilio"],
        "Lugar propio de instalación - Localidad": df["Instalación - Localidad"],
        "Lugar propio de instalación - Provincia": df["Instalación - Provincia"],

        "Instrumento verificado": "Balanza para pesar camiones",  # o tomamos 'Balanza tipo 1' si preferís
        "Fabricante receptor": df["Fabricante receptor"],
        "Marca Receptor": df["Marca Receptor"],
        "Modelo Receptor": df["Modelo Receptor"],
        "N° de serie Receptor": df["N° de serie Receptor"],
        "Cód ap. mod. Receptor": df["Cód ap. mod. Receptor"],
        "Origen Receptor": df["Origen Receptor"],

        "e": df["e"],
        "máx": df["máx"],
        "mín": df["mín"],
        "dd=dt": df["dd=dt"],
        "clase": df["clase"],

        "Tipo (Indicador)": "electrónica",  # fijo como en tu hoja
        "Marca Indicador": df["Marca Indicador"],
        "Modelo Indicador": df["Modelo Indicador"],
        "N° de serie Indicador": df["N° de serie Indicador"],
        "Código Aprobación (Indicador)": df["Código ap. mod. Indicador"],
        "Origen Indicador": df["Origen Indicador"],
        "N° de Aprobación Modelo (Indicador)": df["N° Aprobación Modelo (Ind.)"],
        "Fecha de Aprobación Modelo (Indicador)": df["Fecha Aprobación Modelo (Ind.)"],

        "Tipo de Verificación": df["Tipo de Verificación"],
        "Tolerancia": df["Tolerancia"],
        "CUIT del solicitante": df["CUIT"],
    })
    return out

def exportar_verificacion(df_verif: pd.DataFrame, ruta_xlsx: str) -> str:
    if not ruta_xlsx.lower().endswith(".xlsx"):
        ruta_xlsx += ".xlsx"
    with pd.ExcelWriter(ruta_xlsx, engine="xlsxwriter") as writer:
        df_verif.to_excel(writer, sheet_name="Verificación", index=False)
        ws = writer.sheets["Verificación"]
        # formato simple
        for i, col in enumerate(df_verif.columns):
            ws.set_column(i, i, 28)
    return ruta_xlsx


Usuario y contraseña

In [ ]:
import getpass

USUARIO = input("Usuario INTI: ").strip()
PASSWORD = getpass.getpass("Contraseña INTI: ").strip()
OT = input("N° de OT (formato 307-xxxxx): ").strip()

df_cam = await extraer_camiones_por_ot_async(OT, USUARIO, PASSWORD, mostrar_navegador=True)
df_ver = armar_hoja_verificacion(df_cam)
ruta = exportar_verificacion(df_ver, f"OT_{OT}_VERIFICACION_PREVIA.xlsx")
print("Archivo generado:", ruta)

df_ver.head(3)
